In [11]:

import org.apache.spark.sql.Row
import org.apache.spark.sql.types._


val flatfile = sc.textFile("skip_first_3_rows_from_text_file.txt")

val flatfileWithDroppedLines = flatfile.mapPartitionsWithIndex{ (id_x, iter) => if(id_x==0) iter.drop(4) else iter }
// iter.drop(4) as 4th row is a header and first 3 rows have incorrect data


flatfileWithDroppedLines.collect()
// Array[String] = Array(1,Jer, 2,Tom, 3,Dan)

import org.apache.spark.sql.Row
import org.apache.spark.sql.types._
flatfile: org.apache.spark.rdd.RDD[String] = skip_first_3_rows_from_text_file.txt MapPartitionsRDD[23] at textFile at <console>:48
flatfileWithDroppedLines: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[24] at mapPartitionsWithIndex at <console>:50
res4: Array[String] = Array(1,Jer, 2,Tom, 3,Dan)


In [46]:
val schema = StructType(Array(
    StructField("id", IntegerType),
    StructField("name", StringType)
))

val df = spark.read
  .format("com.databricks.spark.csv")
  .option("header", "false")
  .option("inferSchema", "false")
  .option("delimiter", ",")
  //.option("mode", "DROPMALFORMED") this alone will drop incorrect record, no need to check for isNotNull later
  .schema(schema)
  .load("skip_first_3_rows_from_text_file.txt")

df.show(false)


+---+----+
|id |name|
+---+----+
|1  |Jer |
|2  |Tom |
|3  |Dan |
+---+----+



schema: org.apache.spark.sql.types.StructType = StructType(StructField(id,IntegerType,true),StructField(name,StringType,true))
df: org.apache.spark.sql.DataFrame = [id: int, name: string]


In [44]:
df.filter($"id".isNotNull).show(false)

+---+----+
|id |name|
+---+----+
|1  |Jer |
|2  |Tom |
|3  |Dan |
+---+----+



In [ ]:
// PySpark equivalent

// import csv

// customSchema = StructType([ \
//     StructField("Col1", StringType(), True), \
//     StructField("Col2", StringType(), True)])

// df = sc.textFile("file.csv")\
//         .mapPartitions(lambda partition: csv.reader([line.replace('\0','') for line in partition],delimiter=',', quotechar='"')).filter(lambda line: len(line) > 2 and line[0] != 'Col1')\
//         .toDF(customSchema)
        
        